# **BERT Model Domain Adaptation Testing**

In [1]:
from transformers import BertForMaskedLM, BertTokenizer

tokenizer = BertTokenizer.from_pretrained('beomi/kcbert-base')
model = BertForMaskedLM.from_pretrained('beomi/kcbert-base')

/Users/choejaehun/anaconda3/envs/ssis/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
from datasets import load_dataset

# local file을 불러오기 위해선 ('형식', '경로')를 arg로 넣어야함.
# csv 외에도 json, text, parquet, sql 형식을 지원

train = load_dataset('csv',data_files='data/book_train_128.csv')
validation = load_dataset('csv',data_files='data/book_validation_128.csv')

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 397.90it/s]
Generating train split: 175900 examples [00:00, 397656.93 examples/s]
Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 318.16it/s]
Generating train split: 880 examples [00:00, 104809.96 examples/s]


In [4]:
train['train'][0]

{'Unnamed: 0': 0,
 'sen': '이 책의 특징ㆍ코딩의 기초 기초수학 논리의 기초 C언어의 기초 순서도 개념ㆍ16장으로 구성하여 34시간 정도의 공부ㆍ유튜브 동영상 강의ㆍ블로그에서 모든 예제파일 다운로드 및 소프트웨어 교육 참조ㆍ공개 소프트웨어무료 컴파일러를 사용 이 책의 대상 독자ㆍ중학생 고등학생 컴퓨터관련 비전공 대학생ㆍIT 관련 취업 교육생ㆍ정보처리 기능사 산업기사 실기문제알고리즘 준비 이 책에서 다루는 내용ㆍ순서도 개념 및 코딩 작성ㆍ기초수학'}

In [5]:
def tokenize_function(examples):
    return tokenizer(examples['sen'], max_length=128, padding=True, truncation=True)

train_data_set = train['train'].map(tokenize_function)
validation_data_set = validation['train'].map(tokenize_function)

Map: 100%|██████████| 880/880 [00:00<00:00, 5883.34 examples/s]


In [6]:
print(train_data_set[0])

{'Unnamed: 0': 0, 'sen': '이 책의 특징ㆍ코딩의 기초 기초수학 논리의 기초 C언어의 기초 순서도 개념ㆍ16장으로 구성하여 34시간 정도의 공부ㆍ유튜브 동영상 강의ㆍ블로그에서 모든 예제파일 다운로드 및 소프트웨어 교육 참조ㆍ공개 소프트웨어무료 컴파일러를 사용 이 책의 대상 독자ㆍ중학생 고등학생 컴퓨터관련 비전공 대학생ㆍIT 관련 취업 교육생ㆍ정보처리 기능사 산업기사 실기문제알고리즘 준비 이 책에서 다루는 내용ㆍ순서도 개념 및 코딩 작성ㆍ기초수학', 'input_ids': [2, 2451, 2856, 4042, 1, 12755, 12755, 4110, 4087, 9109, 4042, 12755, 36, 4151, 4071, 4042, 12755, 21117, 4029, 1, 17474, 8455, 23335, 8432, 16502, 1, 13973, 1, 8229, 2289, 4231, 4129, 4046, 17341, 4091, 4273, 1476, 1895, 22181, 5301, 4071, 8614, 1, 1895, 22181, 5301, 4071, 4211, 4018, 3015, 4129, 4046, 4053, 4180, 9021, 2451, 2856, 4042, 12158, 1, 16264, 15513, 11315, 1664, 4203, 4239, 1, 9081, 10092, 1, 16763, 4107, 11240, 8649, 2009, 4184, 8739, 12325, 12261, 9242, 2451, 2856, 7971, 17140, 4008, 1, 10070, 1476, 3044, 4389, 1, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [7]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="test_trainer",
    per_device_eval_batch_size=64,
    per_device_train_batch_size=64,
    logging_steps=100,
    num_train_epochs=2,
    evaluation_strategy='epoch',
)

In [8]:
from transformers import DataCollatorForLanguageModeling
data_collator_BERT = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15, return_tensors="pt"
)

In [9]:
from transformers import TrainerCallback

# custom callback 만들기, 이때 TrainerCallback을 상속 받아야함.
class myCallback(TrainerCallback):
  def on_step_begin(self, args, state, control, logs=None, **kwargs):
    # step은 1회 batch 진행을 의미함. step의 시작일 때 아래의 내용을 실행
      if state.global_step % args.logging_steps == 0:
        # state는 현재 step, epoch 등 진행 상태에 대한 값을 불러옴
        # arg는 훈련 옵션으로 설정한 값을 불러옴.
          print("")
          print(
              f"{int(state.epoch)}번째 epoch 진행 중 --- {state.global_step}번째 step 결과"
          )

In [10]:
from transformers import Trainer

class customtrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    ############# 내용 추가
    def step_check(self):
        # state는 현 상태를 담는 attribute임.
        return self.state.global_step
    
    ### Training 단계에서 
    def compute_loss(self, model, inputs, return_outputs=False):
        """
        How the loss is computed by Trainer. By default, all models return the loss in the first element.

        Subclass and override for custom behavior.
        """
        if self.label_smoother is not None and "labels" in inputs:
            labels = inputs.pop("labels")
        else:
            labels = None
        outputs = model(**inputs)
        # Save past state if it exists
        # TODO: this needs to be fixed and made cleaner later.
        if self.args.past_index >= 0:
            self._past = outputs[self.args.past_index]

        if labels is not None:
            loss = self.label_smoother(outputs, labels)
        else:
            # We don't use .loss here since the model may return tuples instead of ModelOutput.
            loss = outputs["loss"] if isinstance(outputs, dict) else outputs[0]

        ############# 내용 추가
        if self.step_check() % self.args.logging_steps == 0:
            # step_check = 현 step 파악
            # args.logging_steps = argument에서 지정한 step 불러오기

            # batch 중 0 번째 위치한 문장 선택
            num = 1
            input_id = inputs.input_ids[num].reshape(-1).data.tolist()
            output_id = outputs.logits[num].argmax(dim=-1).reshape(-1).data.tolist()
            attention_mask = inputs.attention_mask[num]

            # mask가 위치한 idx 추출하기
            mask_idx = (inputs.input_ids[num] == 4).nonzero().data.reshape(-1).tolist()

            # padding 제거
            input_id_without_pad = [
                input_id[i] for i in range(len(input_id)) if attention_mask[i]
            ]
            output_id_without_pad = [
                output_id[i] for i in range(len(output_id)) if attention_mask[i]
            ]

            # id to token
            # [1:-1] [CLS,SEP] 제거
            inputs_tokens = self.tokenizer.convert_ids_to_tokens(input_id_without_pad)[
                1:-1
            ]
            outputs_tokens = self.tokenizer.convert_ids_to_tokens(
                output_id_without_pad
            )[1:-1]

            # output mask 부분 표시하기
            for i in mask_idx:
                # [CLS,SEP 위치 조정]
                outputs_tokens[i - 1] = "[" + outputs_tokens[i - 1] + "]"

            inputs_sen = self.tokenizer.convert_tokens_to_string(inputs_tokens)
            outputs_sen = self.tokenizer.convert_tokens_to_string(outputs_tokens)

            print(f"input 문장 : {''.join(inputs_sen)}")
            print(f"output 문장 : {''.join(outputs_sen)}")

        return (loss, outputs) if return_outputs else loss

In [11]:
trainer = customtrainer(
    model=model,
    train_dataset=train_data_set,
    eval_dataset=validation_data_set,
    data_collator=data_collator_BERT,
    args=training_args,
    tokenizer=tokenizer,
    callbacks=[myCallback],
)

trainer.train()

# 학습 중단된 시점부터 다시 시작
# trainer.train('test_trainer/checkpoint-500)

  0%|          | 0/5498 [00:00<?, ?it/s]


0번째 epoch 진행 중 --- 0번째 step 결과
input 문장 : 또한 다른 분야이지만 [MASK] [MASK] [MASK]할 때 알아두면 유용한 내용 필자가 실무 [MASK] 종사하면서 체득한 유용한 팁 등도다니며고 있습니다
output 문장 : 또한 다른 분야이지만 [,] [##을] [근무]할 때 알아두면 유용한 내용 필자가 실무 [##에] 종사하면서 체득한 유용한 내용 등도 말해주고 있습니다


  2%|▏         | 100/5498 [07:30<6:46:15,  4.52s/it]

{'loss': 0.1539, 'learning_rate': 4.90905783921426e-05, 'epoch': 0.04}

0번째 epoch 진행 중 --- 100번째 step 결과
input 문장 : 혹시나 파이썬 설치하기 싫어요 하는 나같은 독자가 또 있다면 쥬피터 노트북 [MASK] . .
output 문장 : 혹시나 파이썬 설치하기 싫어요 하는 나같은 독자가 또 있다면 코피터 노트북 [.] . .


  3%|▎         | 180/5498 [12:05<5:57:14,  4.03s/it]

RuntimeError: MPS backend out of memory (MPS allocated: 8.48 GB, other allocations: 18.68 GB, max allowed: 27.20 GB). Tried to allocate 46.51 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).